In [1]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise RuntimeError("missing key")

In [ ]:
# do not run in VS code
# Setup for Colab

# set GOOGLE_API_KEY in Colab secrets
# the below retrieves the set key from the Colab
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

# helper function to be used 

# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")


In [3]:

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

**Centralized model + factory for multi-agent systems**

When defining multiple agents that use the same LLM (e.g., Gemini), use a central constant or small factory so you don't repeat literals — and prefer a factory (make_gemini) so you can easily vary `model`, `temperature`, or `retry options` per agent. 

*Avoid sharing the same live LLM instance if the SDK docs warn of internal state — use the factory to create instances.*


- Single SOURCE OF TRUTH for model name & retry config.
- Factory = easy per-agent overrides (critique vs drafting).
- Centralize deterministic settings (temperature, max_tokens) in one place.
- Avoid repeating code and typos.

In [ ]:
# Centralized model config + factory
MODEL_NAME = "gemini-2.5-flash-lite"

def make_gemini(model_name: str = MODEL_NAME, retry_options = retry_config):
    # return a fresh Gemini instance so each agent can have its own instance if needed
    return Gemini(model=model_name, retry_options=retry_options)

# default instance (or call make_gemini() per-agent)
GEMINI_DEFAULT = make_gemini()
# use GEMINI_DEFAULT instead of repeating Gemini(...)

In [ ]:



# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=GEMINI_DEFAULT,
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

#----------------------------------

# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=GEMINI_DEFAULT,
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

#----------------------------------
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=GEMINI_DEFAULT,
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

#-------------------
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=GEMINI_DEFAULT,
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")




✅ research_agent created.
✅ summarizer_agent created.
✅ root_agent created.
✅ root_agent created.


In [6]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest advancements in agentic AI in Finance?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in agentic AI in Finance?


ResearchCoordinator > Here's a summary of the key advancements in agentic AI in Finance:

*   **Significant Market Growth:** The agentic AI market in finance is predicted to expand dramatically, from $2.1 billion in 2024 to $81 billion by 2034, driven by technological advancements.
*   **Automation & Efficiency Gains:** Agentic AI automates tasks like data entry, compliance checks, and transaction processing, leading to increased productivity and reduced errors in areas such as accounting and fraud detection.
*   **Enhanced Decision-Making & Risk Management:** AI agents improve credit risk assessment by using real-time data and enable autonomous market monitoring and risk mitigation in trading.
*   **Hyper-Personalized Customer Experiences:** Agentic AI facilitates personalized financial management and adaptive tools tailored to individual customer goals and risk tolerance.
*   **Streamlined Regulatory Compliance & Autonomous Operations:** AI agents automate compliance monitoring and o

## Sequential Workflows




**Best practices **

- **Use placeholders**: `{topic}`, `{journal_guidelines}` in the instruction.
Pass concrete values in the inputs to the runner / Crew kickoff.
- Include a short, explicit checklist of journal constraints in `{journal_guidelines}` (word limits, section order, citation style, figure/table rules).
- Keep `temperature` low for deterministic outlines (e.g., 0–0.3).
- Validate the returned outline against the checklist and loop if missing constraints.
- Use `Process.sequential` / rate-limit handling as you already do with retry_config.

*Use a template (placeholders) in the agent instruction and pass the concrete values (topic, journal guidelines) via the runner / kickoff inputs. 
This keeps the agent prompt generic and lets you reuse it across topics and journals.*




In [ ]:
# Centralized model config + factory
MODEL_NAME = "gemini-2.5-flash-lite"

def make_gemini(model_name: str = MODEL_NAME, retry_options = retry_config):
    # return a fresh Gemini instance so each agent can have its own instance if needed
    return Gemini(model=model_name, retry_options=retry_options)

# default instance (or call make_gemini() per-agent)
GEMINI_DEFAULT = make_gemini()
# use GEMINI_DEFAULT instead of repeating Gemini(...)

#-----------------------------

planning_agent = Agent(
    name="PlanningAgent",
    model=GEMINI_DEFAULT,
    instruction="""Create an outline for an academic paper on a given topic.
Requirements:
- Provide a catchy title
- A short abstract (50-100 words)
- 5 keywords
- 3–5 main sections (e.g., Introduction, Background, Methodology, Results/Discussion), each with bullet points of content
- A concluding thought suggesting an initial search query
- Apply formatting rules as specified in the provided editorial guidelines 
""",
    output_key="paper_outline",
)
print("✅ PlanningAgent created.")

#-----------------------------

research_agent = Agent(
    name="ResearchAgent",
    model=GEMINI_DEFAULT,
    instruction="""Use google_search tool to find relevant papers on Google Scholar
    Requirements:
    - Papers should be published within the last 2 years
    - Be in English""",
    tools=[google_search],
    output_key="research_findings",
)
print("✅ ResearchAgent created.")

#-----------------------------

# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model=GEMINI_DEFAULT,
    # The `{paper_outline}` and {research_findings} placeholders automatically inject the state value from the previous agent's output.
    instruction="""Following this outline strictly: {paper_outline}
    Use {research_findings}
    Write a brief, 200-word  research proposal""",
    output_key="paper_draft",  # The result of this agent will be stored with this key.
)
print("✅ WriterAgent created.")

#-----------------------------

# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model=GEMINI_DEFAULT,
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {paper_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  # This is the final output of the entire pipeline.
)

print("✅ EditorAgent created.")
#------------------

root_agent = SequentialAgent(
    name="PaperPipeline",
    sub_agents=[planning_agent, research_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ PlanningAgent created.
✅ ResearchAgent created.
✅ WriterAgent created.
✅ EditorAgent created.
✅ Sequential Agent created.


In [8]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a research brief about AI uses in Finance, abstract should be less than 150 words")


 ### Created new session: debug_session_id

User > Write a research brief about AI uses in Finance, abstract should be less than 150 words
PlanningAgent > ## The Algorithmic Ascent: Navigating the Landscape of Artificial Intelligence in Modern Finance

**Abstract:**

This paper presents an integrative literature review exploring the multifaceted applications of Artificial Intelligence (AI) within the financial sector. Driven by vast datasets and computational power, AI is revolutionizing traditional financial practices, from algorithmic trading and risk management to customer service and fraud detection. This review synthesizes existing research to map the current state of AI adoption, identify key technological drivers, and highlight emerging trends. It aims to provide a comprehensive overview of AI's impact, examining both its transformative potential and the inherent challenges, such as ethical considerations and regulatory hurdles.

**Keywords:** Artificial Intelligence, Finance, 

In [ ]:
## Parallel Workflows

In [ ]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends in psychology. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [10]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [ ]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current trends in economy. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [12]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [13]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [17]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily research briefing on Tech, Health, and Finance"
)


 ### Created new session: debug_session_id

User > Run the daily research briefing on Tech, Health, and Finance
InitialWriterAgent > The crimson sunrise bled across the smog-choked sky, a familiar sight for Anya. Her implant buzzed, a silent notification of the daily briefing. Tech: Neural interfaces saw a 12% surge in adoption, promising seamless data transfer. Health: Gene-editing breakthroughs in regenerative medicine offered hope for chronic conditions, but ethical debates raged. Finance: The global digital currency market stabilized, with a cautious optimism following last week's crash. Anya sighed, the weight of the world’s progress pressing down. She logged off, the sterile hum of her apartment a stark contrast to the vast, complex world she’d just glimpsed. Another day, another deluge of information.
CriticAgent > Here are a few suggestions to strengthen your story:

1.  **Show, Don't Just Tell Anya's Reaction:** Instead of stating Anya sighed and felt the "weight of the world

_ResourceExhaustedError: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 6.067216737s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_

## Loop Workflows - The Refinement Cycle

In [ ]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a research proposal (around 100-150 words).
    Output only the text of the draft, with no introduction or explanation.""",
    output_key="current_draft",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}


print("✅ exit_loop function created.")


# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")


# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")


✅ initial_writer_agent created.
✅ critic_agent created.
✅ exit_loop function created.
✅ refiner_agent created.
✅ Loop and Sequential Agents created.


In [16]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > The salt spray kissed Silas’s weathered face as he climbed the winding stairs, the oil lamp casting dancing shadows. For fifty years, the lighthouse had been his world, its beam his only constant companion. Tonight, however, a faint, ethereal glow emanated from his battered sea chest. Puzzled, he lifted the heavy lid.

Nestled amongst old charts and barnacle-encrusted tools lay a vellum map, unlike any he’d ever seen. Strange, swirling symbols pulsed with a soft, cerulean light, illuminating intricate coastlines and unknown islands. A single, glowing 'X' marked a spot far out in the uncharted abyss. Silas, his heart pounding with a thrill he hadn't felt since his youth, traced the luminous lines with a trembling finger. The sea, his lifelong mistress, had just whispered a new secret.
CriticAgent > The story is a good start, creat

_ResourceExhaustedError: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 33.473432541s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free